In [1]:
class FuzzyInput:
    def __init__(self, name, xs, ys, x0):
        self.name = name
        self.xs = xs
        self.ys = ys
        self.mu = self.calcMu(x0)
        
    def calcMu(self, x0):
        if x0 < self.xs[0]:
            return self.ys[0]
        elif x0 > self.xs[-1]:
            return self.ys[-1]
        
        for i, (x1, x2) in enumerate(zip(self.xs[:-1], self.xs[1:])):
            if x0 >= x1 and x0 < x2:
                return FuzzyInput.line_through_two_points(x1, self.ys[i], x2, self.ys[i+1])(x0)
    
    @staticmethod
    def line_through_two_points(x1, y1, x2, y2):
        return lambda x: y1 + (y2 - y1)/(x2 - x1) * (x - x1)

In [2]:
class FuzzyOutput:
    def __init__(self, name, xs, ys):
        self.name = name
        self.xs = xs
        self.ys = ys
        self.mu = 0
        cs = [x for x, y in zip(xs, ys) if y == 1]
        self.c = sum(cs) / len(cs)

In [5]:
import enum

class LogicOperator(enum.Enum):
    AND = enum.auto()
    OR = enum.auto()

In [17]:
class Rule:
    def __init__(
        self,
        operand1: FuzzyInput,
        operand2: FuzzyInput,
        output: FuzzyOutput,
        operator: LogicOperator
    ):
        
        if operator == LogicOperator.AND:
            output.mu = max(output.mu, min(operand1.mu, operand2.mu))
        elif operator == LogicOperator.OR:
            output.mu = max(output.mu, max(operand1.mu, operand2.mu))

In [18]:
potrosnja = []
potrosnja.append(FuzzyInput('mala', [3,10],[1,0],9))
potrosnja.append(FuzzyInput('srednja', [7,10,12,15],[0,1,1,0],9))
potrosnja.append(FuzzyInput('velika', [12,15],[0,1],9))

pouzdanost = []
pouzdanost.append(FuzzyInput('visoka', [5,10],[1,0], 8))
pouzdanost.append(FuzzyInput('niska', [8,15],[0,1], 8))

vrednost = []
vrednost.append(FuzzyOutput('mala', [7,15],[1,0]))
vrednost.append(FuzzyOutput('srednja', [7,15,25,40],[0,1,1,0]))
vrednost.append(FuzzyOutput('velika', [25,40],[0,1]))

rules = []
rules.append(Rule(potrosnja[0], pouzdanost[0], vrednost[2], LogicOperator.AND))
rules.append(Rule(potrosnja[0], pouzdanost[1], vrednost[1], LogicOperator.AND))
rules.append(Rule(potrosnja[1], pouzdanost[0], vrednost[1], LogicOperator.AND))
rules.append(Rule(potrosnja[1], pouzdanost[1], vrednost[1], LogicOperator.AND))
rules.append(Rule(potrosnja[2], pouzdanost[0], vrednost[1], LogicOperator.AND))
rules.append(Rule(potrosnja[2], pouzdanost[1], vrednost[0], LogicOperator.AND))

In [19]:
for i in range(len(vrednost)):
    print(vrednost[i].mu)

0
0.3999999999999999
0.1428571428571429


In [22]:
import numpy as np

In [23]:
def defuzzify():
    cs = np.array([v.c for v in vrednost])
    mus = np.array([v.mu for v in vrednost])
    
    return cs.dot(mus) / mus.sum()

In [24]:
defuzzify()

25.263157894736846